In [1]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import pandas as pd
import torch
import os
import glob
import wandb
import ultralytics
import cv2
print('ultralytics %s' % ultralytics.__version__)
print('wandb %s' % wandb.__version__)
from wandb.integration.ultralytics import add_wandb_callback

c:\Users\johna\anaconda3\envs\fyp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


ultralytics 8.3.75
wandb 0.19.6


In [2]:
!nvidia-smi

Tue Feb 18 11:40:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   46C    P0             29W /  125W |     818MiB /   6144MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Arguments setting for the model

In [3]:
args = dict(
    data = "data.yaml",
    epochs = 1,
    patience = 5,
    batch = 4,
    save = False,
    save_period = 5,
    device = '0',
    workers = 1,
    project = "weights",
    name = "try",
    exist_ok = False, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    conf = 0.01,
    iou = 0.6,
    degrees = 0.0, 
    shear = 0.0,
    perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

Loading the model yoloV8 and running wandb for logging

Training of model

In [4]:
settings.update({"wandb": True})
wandb.login()

wandb: Currently logged in as: ejento47 (object-detection-glottis-fyp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
wandb.init(project='YOLOv8-demo', name='run1')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
# for loading model
model = YOLO('yolov8n.pt')

In [7]:
add_wandb_callback(model, enable_model_checkpointing=True)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [8]:
#training model
results = model.train(cfg="train_cfg.yaml")

Ultralytics 8.3.75  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=5, batch=4, imgsz=640, save=False, save_period=5, cache=False, device=0, workers=1, project=weights, name=try2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.6, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

train: Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]
val: Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]


Plotting labels to weights\try2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to weights\try2
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      1.11G      1.451      2.975      1.216          4        640: 100%|██████████| 1185/1185 [01:49<00:00, 10.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:20<00:00, 16.54it/s]


                   all      10665       9969      0.808      0.783      0.798       0.51
Ultralytics 8.3.75  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:09<00:00, 19.20it/s]


                   all      10665       9969      0.882      0.871      0.885      0.696
               Glottis       9969       9969      0.882      0.871      0.885      0.696


Generating Visualizations for batch-1/1: 100%|██████████| 8/8 [00:00<00:00, 22.70it/s]

Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to weights\try2


lr/pg0,▃▆█▇▆▅▄▃▂▁
lr/pg1,▃▆█▇▆▅▄▃▂▁
lr/pg2,▃▆█▇▆▅▄▃▂▁
metrics/mAP50(B),▁▂▄▆▆▇▆▇██
metrics/mAP50-95(B),▁▂▃▄▅▆▆▇██
metrics/precision(B),▁▂▅▇▆▆█▆██
metrics/recall(B),▁▁▃▄▆▆▇▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▄▃▃▂▂▁


In [9]:
model_path = 'yolov8nDemo.pt'
model.save(model_path)

In [13]:
# wandb.log_model(model_path)
# wandb.log({"val_loss": results[0], "val_mAP": results[1]})

In [14]:

wandb.finish()

Running Predictions on model using bagls video and dhs video

Can use OpenCV for better video processing and visualization of the predictions

In [17]:
dhs_video = r'D:\FYP Code\testfiles\dhs.mp4'
baglsC_video = r'D:\FYP Code\testfiles\bagls-color.mp4'
bagls_video = r'D:\FYP Code\testfiles\bagls.mp4'
test_path = r'D:\FYP Datasets\fyp\training\test\images'
weight_path = r'D:\FYP Code\weights\try2\weights\best.pt'

In [18]:
model = YOLO(weight_path)
# model.predict(source= test_path, save_dir = 'results', save= True)
model.predict(source=dhs_video, save_dir = 'results', save= True)
model.predict(source=baglsC_video, save_dir = 'results', save= True)
model.predict(source=bagls_video, save_dir = 'results', save= True)




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 7.6ms
video 1/1 (frame 2/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.0ms
video 1/1 (frame 3/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 9.7ms
video 1/1 (frame 4/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.0ms
video 1/1 (frame 5/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.7ms
video 1/1 (frame 6/808) D:\F

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Torus Tobarius', 1: 'Soft palate', 2: 'Uvula', 3: 'Glottis', 4: 'True vocal folds', 5: 'Epiglottis'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [2, 2, 2],
         [2, 2, 2]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
     

In [ ]:
# wandb.log({"final_mAP": results.metrics.mAP50})
# wandb.finish()